In [1]:
%matplotlib inline

In [2]:
import math
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import seaborn as sns
from IPython.display import display, HTML
from sklearn.cross_decomposition import CCA
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, NoiseTypes, DetrendOperations, WindowOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

In [3]:
board_id = BoardIds.UNICORN_BOARD
board = BoardShim(board_id, BrainFlowInputParams())

In [4]:
channels = board.get_eeg_channels(board_id) #EEG Channels
timestamp_channel = board.get_timestamp_channel(board_id) # Timestamp channel
marker_channel = board.get_marker_channel(board_id) # Marker channel for synchronization
sampling_rate = BoardShim.get_sampling_rate(board_id) # Hz
print("Channels: ", channels)
print("Timestamp: ", timestamp_channel)
print("Marker: ", marker_channel)
print("Sampling Rate: ", sampling_rate)

Channels:  [0, 1, 2, 3, 4, 5, 6, 7]
Timestamp:  17
Marker:  18
Sampling Rate:  250


In [6]:
df = pd.read_csv('data/eeg_data2.csv', header=None)

In [7]:
x_ = df[timestamp_channel]
Oz = df[channels[6]]
Po7 = df[channels[5]]
Po8 = df[channels[7]]

montage_1 = Po7.to_numpy() - Oz.to_numpy()
montage_2 = Po8.to_numpy() - Oz.to_numpy()

In [8]:
filtered_eeg = [montage_1, montage_2]

for i in range(len(filtered_eeg)):
    DataFilter.perform_bandpass(filtered_eeg[i], sampling_rate, 2.0, 30.0, 2, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)
    DataFilter.perform_bandstop(filtered_eeg[i], sampling_rate, 58.0, 62.0, 5, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)

In [22]:
first = df.loc[df[marker_channel] == 4].index[0]
first_eeg = []
for i in range(2):
    first_eeg.append(filtered_eeg[i][first:first+sampling_rate*15])
target_time_18hz = x_[first:first+sampling_rate*15]

In [23]:
f_18 = 18
f_16 = 16
f_24 = 24
f_9 = 9

y_18 = []
y_16 = []
y_24 = []
y_9 = []

# time = np.linspace(0, 10, sampling_rate*15)
time = target_time_18hz
for i in range(2):
    y_18.append(np.sin(2*(i+1)*np.pi*f_18*time))
    y_18.append(np.cos(2*(i+1)*np.pi*f_18*time))
    y_16.append(np.sin(2*(i+1)*np.pi*f_16*time))
    y_16.append(np.cos(2*(i+1)*np.pi*f_16*time))
    y_9.append(np.sin(2*(i+1)*np.pi*f_9*time))
    y_9.append(np.cos(2*(i+1)*np.pi*f_9*time))
    y_24.append(np.sin(2*(i+1)*np.pi*f_24*time))
    y_24.append(np.cos(2*(i+1)*np.pi*f_24*time))
    
# print(y_18)
# plt.plot(time, y_18[0])

In [24]:
def coeff(x, y):
    x_t = np.transpose(x)
    y_t = np.transpose(y)
    
    cca = CCA(n_components=2)
    cca.fit_transform(x_t, y_t)
    X_c, Y_c = cca.transform(x_t, y_t)
    corr=[]
    
    for i in range(0,2):
	    corr.append(np.corrcoef(X_c[:,i], Y_c[:,i])[0,1])
    
    return max(corr)

In [25]:
results = []

results.append(coeff(first_eeg, y_18))
results.append(coeff(first_eeg, y_16))
results.append(coeff(first_eeg, y_9))
results.append(coeff(first_eeg, y_24))

target_f = np.argmax(results)

if target_f == 0:
    print("18")
if target_f == 1:
    print("16")
if target_f == 2:
    print("9")
if target_f == 3:
    print("24")


24
